<img src="../assets/logos/Logo_shoot.png" width=500></img>
# Third Model: Deep Learning
1. problems with the probablity
1. strengths of deep learning
1. intro to CNNs

## Data
We will be using past game date to train out model. First we need to download the data by running: (note the data is constantly being updated as games are played)
```bash
./bin/download_data.sh
```
next copy the sample data file into the mock directory of the code for this model
```bash
cp ./data/data.min.json ./containers/Shoot/cnn/mock/data/data.json
```
You will now be able to test your model training localy 

Next we need to upload the data to s3 so we can train the production model on it. run:
```bash
./bin/upload_data.sh
```

## Code
1. code is in /containers/shoot/cnn


## Configure
Before you can deploy your second model you need to configure the deployment. As described in the [Endpoint Notebook](../Endpoint_Reference.ipynb), you will edit to the deployment configuration in /template/main/sagebuild/shoot.js. you need to edit the following field to the following values

|parameter field|Value|description|
|---|---|
|trainsourcefile|`s3://${AssetBucket}/${AssetPrefix}/shoot-cnn.tar.gz`|
|hostsourcefile|`s3://${AssetBucket}/${AssetPrefix}/shoot-cnn.tar.gz`|
|hyperparameters.epochs|200|
|hyperparameters.learning_rate|0.005|
|hyperparameters.width|32|
|hyperparameters.depth|8|
|hyperparameters.patience|10|
|hyperparameters.batch_size|1024|
|metrics|[{Name:"Validation",Regex:"Testing loss: (.*?);"},{Name:"Training",Regex:"Training loss: (.*?);"},{Name:"Throughput",Regex:"Throughput=(.*?);"}]|
|traininstancetype|ml.p3.2xlarge|

## Deploy
1. follow instructions from endpoint reference to commit your code and changes and then push your changes to the remote branch
2. tell opperations to deploy your changes

## Athena
1. Go to the [Athena Console](https://console.aws.amazon.com/athena/home?region=us-east-1#query)
1. you are going to use the following query but you need to first get some values. You need the name of the Amazon Glue catalog that holds the game data and the s3 bucket where that data is stored. You can get these values from the opperations teams. Replace <Game-data-catalog> and <Game-data-bucket> with those values. so for example the line `FROM "<Game-data-catalog>"."<Game-data-bucket>"` will look something like `FROM "game-catalog-adafafdad"."game-archive-adfasdfasdf"`
1. Next replace the three occurances of <Team-Name> with your team name. For example, the line `WHEN winner = '<Team-Name>' THEN 1.0` becomes `WHEN winner = 'my-team' THEN 1.0`

```sql
SELECT type,
         sum(won)/(sum(won)+sum(lost)) as won,
         sum(lost)/(sum(won)+sum(lost)) as lost
FROM 
    (SELECT json_extract(session,'$.type') AS type,
        CASE
            WHEN winner = '<Team-Name>' THEN 1.0
            ELSE 0.0
        END AS won,
        CASE
            WHEN winner != '<Team-Name>' THEN 1.0
            ELSE 0.0
        END AS lost
    FROM 
        (SELECT teama.session AS session,
         teama.teamname AS team,
         winner
        FROM "<Game-data-catalog>"."<Game-data-bucket>"
        UNION
        SELECT teamb.session AS session,
         teamb.teamname AS team,
         winner
        FROM "<Game-data-catalog>"."<Game-data-bucket>" )
        WHERE team = '<Team-Name>' )
    GROUP BY  size
```

## Next Steps
try editing hyperparamters to get better results